# Test Bubble Detection model on new test images
- Evaluates performance of custom bubble detection model on new test images
- Calculates various precision, recall values
- Format: JPEG, PNG, GIF, BMP required --> needs to be corrected in xml file if annotation made for image in tif format

# Note:
- COCO Metrics can only detect up to 100 objects

In [1]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [2]:
# indicate custom model & desired checkpoint from training
CUSTOM_MODEL = 'my_centernet_hg104_1024_7'
CUSTOM_CHECKPOINT = 'ckpt-21'
LABEL_MAP_NAME = 'label_map.pbtxt'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
# indicate name of folder of new test images
NEW_TEST_IMGS = 'test'#'supersaturation_0.16_cropIR'
# give unique name for tf record file
NEW_RECORD_FILE_NAME = 'test_maskJGIR.record'
# max allowed detections
max_detect = 500

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations',CUSTOM_MODEL),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'IMAGE_PATH_TRAIN': os.path.join('Tensorflow', 'workspace','images','train'),
    'IMAGE_PATH_TEST': os.path.join('Tensorflow', 'workspace','images',NEW_TEST_IMGS),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL, 'tfliteexport'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
# create pipeline path for new test images
pipeline_path = os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL, NEW_TEST_IMGS)
if not os.path.exists(pipeline_path):
    if os.name == 'posix':
        os.mkdir(pipeline_path)
    if os.name == 'nt':
        os.mkdir(pipeline_path)

# save path name for new pipeline config to files
files['PIPELINE_CONFIG_NEW'] = os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL, NEW_TEST_IMGS,'pipeline.config')

In [6]:
# CHECKPOINT PATH: copy pipeline.config (old) into new config folder
if os.name =='posix':
    !cp {files['PIPELINE_CONFIG']} {files['PIPELINE_CONFIG_NEW']}
if os.name == 'nt':
    !copy {files['PIPELINE_CONFIG']} {files['PIPELINE_CONFIG_NEW']}

In [7]:
# ANNOTATION PATH: create tf.record file for new test images
# ONLY WORKS WITH JPG FORMAT
!python {files['TF_RECORD_SCRIPT']} -x {paths['IMAGE_PATH_TEST']} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], NEW_RECORD_FILE_NAME)} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/my_centernet_hg104_1024_7/test_maskJGIR.record


In [8]:
# Load pipeline config (copy of old config file)
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG_NEW'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [9]:
# MODIFY PIPELINE.CONFIG
# indicate the path to new test image tf record file
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], NEW_RECORD_FILE_NAME)]
# allow max_detect detections
pipeline_config.model.center_net.object_center_params.max_box_predictions = max_detect
pipeline_config.train_config.max_number_of_boxes = max_detect

In [10]:
# save new pipeline config (to model folder)
config_text = text_format.MessageToString(pipeline_config)
# dfile: accessing the file ('wb'= writing to file in binary mode)                                                                                                                                                                                                     
with tf.io.gfile.GFile(files['PIPELINE_CONFIG_NEW'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [11]:
# Load pipeline config (new/modified config file)
configs_test = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG_NEW'])

In [12]:
configs_test

{'model': center_net {
   num_classes: 1
   feature_extractor {
     type: "hourglass_104"
     channel_means: 104.01361846923828
     channel_means: 114.03422546386719
     channel_means: 119.91659545898438
     channel_stds: 73.60276794433594
     channel_stds: 69.89082336425781
     channel_stds: 70.91507720947266
     bgr_ordering: true
   }
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 1024
       max_dimension: 1024
       pad_to_max_dimension: true
     }
   }
   object_detection_task {
     task_loss_weight: 1.0
     offset_loss_weight: 1.0
     scale_loss_weight: 0.10000000149011612
     localization_loss {
       l1_localization_loss {
       }
     }
   }
   object_center_params {
     object_center_loss_weight: 1.0
     classification_loss {
       penalty_reduced_logistic_focal_loss {
         alpha: 2.0
         beta: 4.0
       }
     }
     min_box_overlap_iou: 0.699999988079071
     max_box_predictions: 500
   }
 },
 'train_config': batch_si

# 7. Evaluate the Model
Calculate performance metrics

In [13]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [14]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG_NEW'], paths['CHECKPOINT_PATH'])

In [15]:
# command can be run from prompt
# EVAL FOLDER MUST BE EMPTY BEFORE RUNNING COMMAND!
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_7 --pipeline_config_path=Tensorflow/workspace/models/my_centernet_hg104_1024_7/test/pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_7


In [ ]:
# cd Tensorflow/workspace/models/my_centernet_hg104_1024_5_v3/eval
# tensorboard --logdir=.

In [17]:
# EVAL PATH: move eval file into image set specific folder
eval_path = os.path.join(paths['CHECKPOINT_PATH'],'eval')
eval_file = os.listdir(eval_path)
eval_file_path = os.path.join(eval_path,eval_file[0])
new_eval_file_path = os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL, NEW_TEST_IMGS,'eval', eval_file[0])

if not os.path.exists(new_eval_file_path):
    if os.name == 'posix':
        os.mkdir(new_eval_file_path)
    if os.name == 'nt':
        os.mkdir(new_eval_file_path)

if os.name =='posix':
    !mv {eval_file_path} {new_eval_file_path}
if os.name == 'nt':
    !move {eval_file_path} {new_eval_file_path}

FileNotFoundError: [Errno 2] No such file or directory: 'Tensorflow/workspace/models/my_centernet_hg104_1024_5/test/eval/events.out.tfevents.1643031764.TUE013625.1133607.0.v2'

In [55]:
new_eval_file_path = os.path.join('Tensorflow','workspace','models',CUSTOM_MODEL, NEW_TEST_IMGS,'eval', eval_file[0])

IndexError: list index out of range

# See Evaluation graphically with Tensorboard
- Eval folder: \TFODCourse\Tensorflow\workspace\models\my_ssd_mobnet\eval --> AV, AR
- Train folder: \TFODCourse\Tensorflow\workspace\models\my_ssd_mobnet\train --> losscurve

in prompt: go to directory --> cd Tensorflow/workspace/models/my_centernet_hg104_1024_5_v3/eval

run command: tensorboard --logdir=.
    
open localhost url